In [ ]:
from warnings import filterwarnings
filterwarnings('ignore')

import pandas as pd
import numpy as np

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score, recall_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import StratifiedKFold


from random import sample
from collections import defaultdict
from statistics import median
from pandarallel import pandarallel
import csv
import time
from pathlib import Path

## Data preparation

In [ ]:
# MAKE FVL and RES datasets
for kind in ['fvl', 'res']:
    target='PATIENT'
    cases = pd.read_csv('../data/hcl/{}_cases.csv'.format(kind), index_col=0)
    cases[target] = 'case'
    controls = pd.read_csv("../data/hcl/{}_controls.csv".format(kind), index_col=0)
    controls[target] = 'control'

    df = pd.concat([cases, controls], ignore_index=True, sort=False)
    labels = list(df[target].values)

    # remove ABS from dataset
    cols = [c for c in df.columns if c.lower()[:3] != 'abs']
    df=df[cols]

    df = df[df.select_dtypes([np.number]).columns].dropna(axis='columns')
    non_na_features = list(df)

    df = df[non_na_features]
    df[target] = labels
    df.to_csv('../data/hcl/{}.csv'.format(kind))

## Info based

In [ ]:
def iind(matrix, alpha = 0.5):
    p = alpha
    matrix = matrix[matrix.sum(axis = 1) > 0.0]
    s = matrix.values.sum()
    if s == 0:
        raise Exception('Sum of matrix is equal to 0')
    matrix = matrix / s
    m1 = matrix.sum(axis = 1)
    m2 = matrix.sum(axis = 0)
    outer_p = np.outer(m1, m2) 
    factor = 1 / (p-1)
    s = np.divide(np.power(matrix,p),np.power(outer_p,(p-1))).sum().sum()
    dividend = np.log(s)
    divisor = (np.log(np.sum(np.power(m2,(2-p)))))
    return dividend / divisor  

In [ ]:
def leave_one_out(m_case, m_control, feature_subset, row_num):
    a = m_case.iloc[[row_num],feature_subset]
    b = m_control.iloc[:,feature_subset]
    partial_res = iind(a.append(b, ignore_index=True)) - iind(m_control.iloc[:,feature_subset])
    results = []

    for i, feature_num in enumerate(feature_subset):
        c = feature_subset.copy()
        c.remove(feature_num)
        a1 = m_case.iloc[[row_num], c]
        b1 = m_control.iloc[:, c]
        ind = iind(a1.append(b1, ignore_index=True)) - iind(m_control.iloc[:,c])
        results.append(ind)
    res = np.subtract(np.array([partial_res]),  np.array([results]))
    return res[0]

In [ ]:
def apply_iind(m_case, m_control, features_num, row_num, k):
    feature_subset = sample(range(0, features_num), k)
    try:
        results = leave_one_out(m_case,m_control,feature_subset, row_num)
    except Exception as e:
        results = np.repeat(np.nan, k)

    return np.r_[np.array(feature_subset), results]

In [ ]:
def plot_dict(d, labels, title=""):
    vals = []
    for k,v in d.items():
        vals.append((labels[k], v))
    vals = list(sorted(vals, key=lambda x: x[1], reverse=True))
    vals = list(zip(*vals))
    if len(vals) == 0:
        print("vals = '[]'")
        return
    x = list(vals[0])
    y = list(vals[1])  
    
    plt.figure(figsize=(15,15))
    plt.bar(range(len(y)), y, align='center')
    plt.xticks(range(len(x)), x, rotation=90)
    plt.title(title)
    plt.show()
    return x, y

In [ ]:
def get_score(feature_ind_vals, feature_ind_to_name, all_measurements, plot=True):
    feature_median = dict()
    for feature_ind, vals in feature_ind_vals.items():
        median = np.median(vals)
        if feature_ind not in all_measurements:
            all_measurements[feature_ind] = vals
        else:
            all_measurements[feature_ind].extend(vals)
        feature_median[feature_ind] = median            
            
    if plot:
        plot_dict(feature_median, feature_ind_to_name, "patient")
        
#     imp_list = list(map(lambda x: x[0], sorted(feature_median.items(), key=lambda x: x[1], reverse=True)))

    return feature_median

In [ ]:
def func_(col, m, m2, features_count, i, k):
    return apply_iind(m, m2, features_count, i, k)

def collect_statistics(col, feature_ind_vals, k):
    control_res = col.tolist()
    for row_num, feature_num in enumerate(control_res):
        if row_num == k:
            break
        ind_val = control_res[row_num + k]
        if not np.isnan(ind_val):
            feature_ind_vals[int(feature_num)].append(ind_val)

In [ ]:
pandarallel.initialize(progress_bar=False)

In [ ]:
def save_feats(k, patient_id, feats):
    with open('k{}_patient{}.csv'.format(k,patient_ind),'w') as out:
        csv_out=csv.writer(out)
        csv_out.writerow(['feat','score'])
        for row in feats:
            csv_out.writerow(row)
            
def save_names(feats_to_names):
    with open('feat_to_name.csv', 'w') as f:
        w = csv.DictWriter(f, feats_to_names.keys())
        w.writeheader()
        w.writerow(feats_to_names)

In [ ]:
def collect_ind_to_name(m):
    feature_ind_to_name = dict()
    for i, feature_name in enumerate(m.columns):
        feature_ind_to_name[i] = feature_name
    save_names(feature_ind_to_name)
#     print(feature_ind_to_name)
    return feature_ind_to_name

In [ ]:
def info_based(m_cases, m_controls, features_subset_size, iters, subset):
    start = time.time()
    feature_ind_to_name = collect_ind_to_name(m_cases)
    FEATURES_COUNT = m_cases.shape[1]
    print('FEATURES COUNT', FEATURES_COUNT)
    
    all_measurements = dict()
    importance_per_patient=dict()
    for patient_ind, ind_row in enumerate(m_cases.iterrows()):
        # Perform FS algorithm
        print("Starting for k = ", features_subset_size, ", patient = ", ind_row[0],  "(i={})...".format(patient_ind))
        d = pd.DataFrame(-1, index=np.arange(2*features_subset_size), columns=[i+1 for i in range(iters)])
        d = d.transpose()
        d = d.parallel_apply(lambda col: func_(col, m_cases, m_controls, FEATURES_COUNT, patient_ind, features_subset_size), axis=1)

        
        #         SAVE dataframe here!!!
        d_to_write = d.to_frame()
        d_to_write.columns = d_to_write.columns.astype(str)
        
        #./date/(fvl|res)/(iterations)/(features_subset_size)/(subset)/patient_id(absolute in whole dataset).
        #i.e. ./18_10_2020/fvl/200000/15/2/13.parquet
        
        path = './{}/{}/{}/{}/{}'.format(DATE, KIND, iters, features_subset_size, subset)
        Path(path).mkdir(parents=True, exist_ok=True)
        d_to_write.to_parquet(path + '/{}.parquet'.format(ind_row[0]))
        print("Result ready for k = ", features_subset_size, ", patient = ", ind_row[0], "...")

        
        # Obtain series from dataframe
        d = pd.read_parquet(path + '/{}.parquet'.format(ind_row[0])).iloc[:,0]
    
        # collect results
        feature_ind_vals = defaultdict(list)
        d.apply(lambda row: collect_statistics(row,feature_ind_vals, features_subset_size))
        feature_median = get_score(feature_ind_vals, feature_ind_to_name, all_measurements)
        
    all_measurements_medians = dict()
    for feature_ind, vals in all_measurements.items():
        median = np.median(vals)
        if median > 0 and median != np.inf:
            all_measurements_medians[feature_ind] = median
    
    selected_fs, iind_val = plot_dict(all_measurements_medians, feature_ind_to_name, "median of all measurements; features_subset_size = " + str(features_subset_size))
    iind_median = np.median(iind_val)
    selected_fs_reduced = list(map(lambda x: x[0],filter(lambda x: x[1] >= iind_median, zip(selected_fs, iind_val))))
    
    if selected_fs_reduced:
        print("Selected features count: ", len(selected_fs_reduced))
        print(selected_fs_reduced)
    else:
        print("selected_fs_reduced was empty for k = ", features_subset_size)        
        
    end = time.time()
    print('Elapsed: ', end - start)

# Feature selection and classification benchmark

In [ ]:
N_SPLITS = 5
RANDOM_STATE = 10
ITERATIONS=50
FEATURES_SUBSET_SIZE=10
TARGET='PATIENT'
CASE='case'
CONTROL='control'
DATE = '18_10_2020'

# CHANGE FOR EACH NOTEBOOK
SUBSET = 0 #in range 0 to N_SPLITS-1
KIND = 'fvl'

In [ ]:
class FeatureSelector:
    """Extracts subset of most informative features and provides rank of all features from train samples."""
    def __init__(self, name):
        self.name = name
        self.features = features
        
    def get_features_rank():
        return self.features_rank
    
    def get_features_subset():
        return self.features_subset

In [ ]:
class RandomForestSelector(FeatureSelector):
    def __init__(self, n_estimators=200):
        super().__init__('RandomForestSelector')
        self.forest = ExtraTreesClassifier(n_estimators=n_estimators, random_state=RANDOM_STATE)
        
    def fit(self, X_train, y_train):
        self.forest.fit(X_train, y_train)
        
    def get_rank(self):
        return self.forest.feature_importances_

In [ ]:
class FeatureSelectorsAggregator:
    """Aggregates partial feature selections obtained from FeatureSelectors."""
    def __init__(self, feature_selectors):
        self.name = 'FeatureSelectorsAggregator'
        self.feature_selectors = feature_selectors
        
    def get_features_rank(self):
        return self.features_rank
    
    def get_features_subset(self):
        return self.features_subset
    
    def fit(self, X_train, y_train):
        for f_selector in self.feature_selectors:
            f_selector.fit(X_train, y_train)
    
    def get_ranks(self):
        ranks = []
        for f_selector in self.feature_selectors:
            ranks.append(f_selector.get_rank())
        return ranks

In [ ]:
class CVEvaluator:
    """Evaluate performance of feature selection and classification model with leave-one-out cross validation
        with n subsets (folds) of the whole dataset. For each iteration find the best classificator
        for current train/test split by performing GridSearchCV on the current train data and asses its performance
        on current test set. The results for each split are averaged and are the approximation of the performance
        of the final model.
            
        If n = 1, the best model is created by selecting features with FeatureSelectorsAggregator
        and performing GridSearchCV on the whole dataset to find the most accurate/sensitive classifier.
        """
    def __init__(self, n_splits, fs_aggregator, labels, features, data, kind):
        """
        Init CVEvaluator.

        Args:
            n_splits: Number of subsets.
            fs_aggregator: FeatureSelectorsAggregator for feature selection at each split.
            labels: list of strings representing labels.
            features: list of feature names. Must match count of colums in data dataframe.
            data: Pandas dataframe containing samples in rows with len(features) columns each.
            kind: healthy, fvl, res.
        """
        self.n_splits = n_splits
        self.fs_aggregator = fs_aggregator
        self.labels = labels
        self.features = features
        self.data = data
        self.kind = kind
        
    def perform_evaluation(self, subset, iterations, features_subset_size):
        skf = StratifiedKFold(n_splits=self.n_splits, random_state=RANDOM_STATE, shuffle=True)
        train_index, test_index = list(skf.split(self.data, self.labels))[subset]
        X_train, X_test = self.data[self.data.index.isin(train_index)], self.data[self.data.index.isin(test_index)]
        y_train, y_test = [self.labels[i] for i in train_index], [self.labels[i] for i in test_index]        
        X_train[TARGET] = y_train

        m_cases = X_train[X_train[TARGET] == CASE]
        m_controls = X_train[X_train[TARGET] == CONTROL]
        del m_cases[TARGET]
        del m_controls[TARGET]
        
        info_based(m_cases, m_controls, features_subset_size, iterations, subset)
            
#             self.fs_aggregator.fit(X_train, y_train)
#             print("Another rank:", len(self.fs_aggregator.get_ranks()))
            

In [ ]:
df = pd.read_csv('../data/hcl/{}.csv'.format(KIND), index_col=0)
labels = list(df[TARGET].values)

del df[TARGET]
features = list(df.columns)

In [ ]:
cv_evaluator = CVEvaluator(N_SPLITS, None, labels, features, df, KIND)
cv_evaluator.perform_evaluation(SUBSET, ITERATIONS, FEATURES_SUBSET_SIZE)